In [1]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import matplotlib.pyplot as plt
import concurrent.futures
import pickle
import warnings
from datetime import date
import hisepy
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed,ProcessPoolExecutor
from tqdm import tqdm
import anndata
import gc
warnings.filterwarnings("ignore")
sc.settings.n_jobs = 60
print("Current working directory:", os.getcwd())

Current working directory: /home/jupyter/BRI_Figures_Final_V2/Figure3/02_UMAP


In [2]:
meta_data_SF4 = pd.read_csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/SF4_meta_data-2024-05-05.csv")

# Read MetaData

In [3]:
meta_data_SF4 = pd.read_csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/SF4_meta_data-2024-05-05.csv")
meta_data_SF4['ageAtEnrollment'] = meta_data_SF4['ageAtEnrollment'].replace('89+', '90')
meta_data_SF4['ageAtEnrollment'] = pd.to_numeric(meta_data_SF4['ageAtEnrollment'], errors='coerce')
meta_data_SF4 = meta_data_SF4.dropna(subset=['ageAtEnrollment'])
meta_data_SF4['Ages'] = meta_data_SF4['ageAtEnrollment']
meta_data_SF4 = meta_data_SF4.drop_duplicates(subset='sample.sampleKitGuid', keep='last')

# Assemble Year 1 Day 0

In [5]:
%%time
file_names= ['/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/SF4/h5ad/'+x+".h5ad" for x in meta_data_SF4['pbmc_sample_id'].tolist()]
adata_list = []
with ThreadPoolExecutor(max_workers=60) as executor:
    future_to_file = {executor.submit(sc.read_h5ad, file_name): file_name for file_name in file_names}
    for future in tqdm(as_completed(future_to_file), total=len(file_names)):
        result = future.result()
        if result is not None:
            adata_list.append(result)

100% 233/233 [09:32<00:00,  2.46s/it] 

CPU times: user 33.9 s, sys: 1min 3s, total: 1min 37s
Wall time: 9min 33s


In [7]:
adata = anndata.concat(adata_list,join='outer')

In [8]:
adata.write_h5ad('SF4_raw.h5ad')

In [4]:
adata=sc.read_h5ad('SF4_raw.h5ad')

In [5]:
#adata.raw=adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [6]:
igl_genes = [gene for gene in adata.var_names if gene.startswith("IGL")]
igk_genes = [gene for gene in adata.var_names if gene.startswith("IGK")]
ighc_genes = [gene for gene in adata.var_names if gene.startswith("IGH")]
exl_genes = igl_genes + igk_genes + ighc_genes

mask = ~adata.var_names.isin(exl_genes)
adata = adata[:, mask]

In [ ]:
sc.pp.highly_variable_genes(adata)
adata = adata[:, adata.var_names[adata.var['highly_variable']]]

In [10]:
%%time
sc.pp.scale(adata)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=50,use_rep='X_pca', n_pcs=30)

CPU times: user 2h 42min 30s, sys: 54min 33s, total: 3h 37min 4s
Wall time: 24min 54s


In [ ]:
%%time
sc.tl.umap(adata,min_dist=0.45,random_state=0)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
adata.write_h5ad('SF4_LogNormalizaed_Scaled.h5ad')